In [152]:
from jax import numpy as jnp
from jax import random, jit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [153]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
train = pd.read_csv('../datasets/bank-note/train.csv', header=None).to_numpy()
test = pd.read_csv('../datasets/bank-note/test.csv', header=None).to_numpy()
### last column is label (-1,1)

x_train = train[:,:-1]
y_train = train[:,-1:]
y_train = np.where(y_train == 0, -1,1)

x_test = test[:,:-1]
y_test = test[:,-1:]
y_test= np.where(y_test == 0, -1,1)

# add column of ones to wrap in b
x_train = np.concatenate((x_train, np.ones((x_train.shape[0],1))), axis=1)
x_test = np.concatenate((x_test, np.ones((x_test.shape[0],1))), axis=1)

shuffle = np.random.choice(len(x_train), len(x_train), replace=False)
x_train = x_train[shuffle]
y_train = y_train[shuffle]


### Logistic Reg model functions that don't change for maximum-a-posteriori (map) vs. maximum likelihood (ml)

In [237]:
def sigmoid(Z):
    A = 1/(1+jnp.exp(-Z))
    cache = Z ## keep for backprop 
    return A, cache

def sigmoid_backprop(dL_dA, cache):
    Z = cache
    s = 1/(1+jnp.exp(-Z))
    dL_dZ = dL_dA * s * (1-s)
    return dL_dZ

def model_forward(X, parameters):
    W, b = parameters['W'], parameters['b']
    Z = X @ W.T + b
    cache =(X,Z)
    preds = 1/(1+jnp.exp(-Z)) # sigmoid
    return preds, cache

def init_params(n_x):
    # n_x -- num input features
    return {
            "W": jnp.zeros((1,n_x)),
            "b": jnp.zeros((1,)),
            }

def update_params(params, grads, lr):
    p = params.copy()
    p['W'] = p['W'] - lr * grads['dL_dW']
    p['b'] = p['b'] - lr * grads['dL_db']                                                                       
    return p

def compute_acc(X, y, params):
    preds,_ = model_forward(X, params)
    return (np.sign(preds) == y).mean()

                     

### Maximum Likelihood 

In [234]:
def model_backprop_ml(preds, y, cache):
    grads = {}
    dL_dA = - ((y / preds) - ((1 - y)/ (1 - preds)))
    X,Z = cache 
    dL_dZ = sigmoid_backprop(dL_dA, Z)
    m = X.shape[1]
    grads["dL_dW"] = (dL_dZ.T @ X) / m
    grads["dL_db"] = dL_dZ.sum(axis=0, keepdims=True) / m
    return grads

ml_objective = lambda y, preds: ((y.T @ np.log(preds)) + ((1-y).T @  np.log(1-preds))) * (-1/len(y))

X = x_train
y = y_train

# @jit
def update_ml(params):
    preds, cache = model_forward(X, params)
    grads = model_backprop_ml(preds, y, cache)
    params = update_params(params, grads, lr)
    return params, preds


#### training

In [ ]:
lr = 0.001
num_epochs = 1000
print_at = 10
params = init_params(X.shape[1])
losses = np.zeros((num_epochs,))

for i in range(num_epochs):
    ### should be able to jit update
    params, preds = update_ml(params)
    loss = bce_loss(preds, y)
    if i % print_at == 0:
        # print(f'{loss=}')
        acc = compute_acc(X,y,params)
        print(acc)
    losses[i] = loss

### Maximum-a-posteriori (map)

In [ ]:
### MAP
def model_backprop_map(preds, y, cache):
    grads = {}
    dL_dA = - ((y / preds) - ((1 - y)/ (1 - preds)))
    X,Z,W = cache 
    dL_dZ = sigmoid_backprop(dL_dA, Z)
    m = X.shape[1]
    grads["dL_dW"] = (dL_dZ.T @ X) / m + (lambd / m)*W ### from regularization term in objective
    grads["dL_db"] = dL_dZ.sum(axis=0, keepdims=True) / m
    return grads


def map_objective(y,preds,W):
    bce_loss = bce_loss(y,preds)
    l2_reg_loss = (W ** 2).sum() * 1/len(y) * (lambd / 2)
    return bce_loss + l2_reg_loss


# @jit
def update_map(params):
    preds, cache = model_forward(X, params)
    cache = (*cache, params['W']) ### need weights for gradients
    grads = model_backprop_map(preds, y, cache)
    params = update_params(params, grads, lr)
    return params, preds


#### training w/ different variances on gaussian prior over weights 

In [231]:
lr = 0.001
num_epochs = 1000
print_at = 10
params = init_params(X.shape[1])
losses = np.zeros((num_epochs,))

variances= jnp.array([0.01, 0.1, 0.5, 1, 3, 5, 10, 100])
lambds = 1 / variances
lambd = 1 / 0.01


for i in range(num_epochs):
    params, preds = update_map(params)
    loss = bce_loss(preds, y)
    if i % print_at == 0:
        print(f'{loss=}')
        acc = compute_acc(X,y,params)
        print(acc)
    losses[i] = loss

0.4461009174311927